In [48]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': [0,0,0,0,1],
    'T': [0,0,0,1,0],
    'L': [0,0,1,0,0],
    'O': [0,1,0,0,0],
    'I': [1,0,0,0,0]
}

X_train = []
Y_train = []
X_test = []
Y_test = []


for key in data['train'].keys():
    X_train.extend(data['train'][key])
    for _ in range(len(data['train'][key])):
        Y_train.append(labels[key])

for key in data['test'].keys():
    X_test.extend(data['test'][key])
    for _ in range(len(data['test'][key])):
        Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

print(X_train.size())
print(Y_train.size())
print(X_test.size())
print(Y_test.size())

torch.Size([800, 3, 3])
torch.Size([800, 5])
torch.Size([200, 3, 3])
torch.Size([200, 5])


In [46]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2)
        self.conv2 = nn.Conv2d(2, 1, 2)
        self.fc = nn.Linear(1, 5)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))        
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.01)

#
# Print number of parameters
#
# for item in list(ConvNet().parameters()):
#     print (item)
print(list(ConvNet().parameters()))
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

get_n_params(ConvNet())

[Parameter containing:
tensor([[[[ 0.0200, -0.3634],
          [-0.1262, -0.1691]]],


        [[[-0.0517, -0.4112],
          [-0.3998,  0.2419]]]], requires_grad=True), Parameter containing:
tensor([-0.0626, -0.3586], requires_grad=True), Parameter containing:
tensor([[[[-0.0118,  0.2424],
          [-0.2287, -0.1248]],

         [[-0.1016,  0.0810],
          [ 0.1141, -0.2180]]]], requires_grad=True), Parameter containing:
tensor([0.2978], requires_grad=True), Parameter containing:
tensor([[ 0.6995],
        [ 0.3544],
        [ 0.2187],
        [-0.6337],
        [-0.9261]], requires_grad=True), Parameter containing:
tensor([-0.6567,  0.6024,  0.9278, -0.9512, -0.5508], requires_grad=True)]


29

In [47]:
# 
# Training
# 
results = {} #{iter: [loss, accuracy]}
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(Y_train)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = X_train[i], Y_train[i]
        inputs = inputs.view(1,1,len(inputs), len(inputs))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(0), labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    if epoch % 10 == 9:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_train[i], Y_train[i]
                images = images.view(1,1,len(images), len(images))

                output = model(images)
                predicted = output.data

                correct += [int(item == predicted[0].max()) for item in predicted[0]] == labels.tolist()
                accuracy = 100. * correct / total
        print('Accuracy of the network: %.2f %%' % (accuracy))

        results[epoch+1] = [running_loss, accuracy]

Epoch: 10, Loss: 14.463
Accuracy of the network: 51.52 %
Epoch: 20, Loss: 14.411
Accuracy of the network: 51.52 %
Epoch: 30, Loss: 13.931
Accuracy of the network: 51.52 %
Epoch: 40, Loss: 12.066
Accuracy of the network: 60.61 %
Epoch: 50, Loss: 10.982
Accuracy of the network: 63.64 %
Epoch: 60, Loss: 10.845
Accuracy of the network: 65.15 %
Epoch: 70, Loss: 10.837
Accuracy of the network: 66.67 %
Epoch: 80, Loss: 10.828
Accuracy of the network: 66.67 %
Epoch: 90, Loss: 10.821
Accuracy of the network: 66.67 %
Epoch: 100, Loss: 10.824
Accuracy of the network: 66.67 %
Epoch: 110, Loss: 10.820
Accuracy of the network: 66.67 %
Epoch: 120, Loss: 10.816
Accuracy of the network: 66.67 %
Epoch: 130, Loss: 10.809
Accuracy of the network: 66.67 %
Epoch: 140, Loss: 10.811
Accuracy of the network: 66.67 %
Epoch: 150, Loss: 10.797
Accuracy of the network: 66.67 %
Epoch: 160, Loss: 10.827
Accuracy of the network: 66.67 %
Epoch: 170, Loss: 10.809
Accuracy of the network: 66.67 %
Epoch: 180, Loss: 10.83